In [59]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(12,10)})

# import sys
# !conda list Check the packages installed

In [60]:
# Loading the cumulative raw data

raw_data_confirmed = pd.read_csv('time_series_19-covid-Confirmed.csv')
raw_data_deaths = pd.read_csv('time_series_19-covid-Deaths.csv')
raw_data_Recovered = pd.read_csv('time_series_19-covid-Recovered.csv')

print("The Shape of Confirmed is: ", raw_data_confirmed.shape)
print("The Shape of Confirmed is: ", raw_data_deaths.shape)
print("The Shape of Confirmed is: ", raw_data_Recovered.shape)

raw_data_confirmed.head()

The Shape of Confirmed is:  (477, 63)
The Shape of Confirmed is:  (477, 63)
The Shape of Confirmed is:  (477, 63)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,59,70,75,82,114,147,177,212,272,322
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,639,639,701,773,839,825,878,889,924,963
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,178,178,200,212,226,243,266,313,345,385
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,149,149,197,238,428,566,673,790,900,1030


In [61]:
# Un-Pivoting the data

raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_Recovered2 = pd.melt(raw_data_Recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])


print("The Shape of Cornfirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornfirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornfirmed is: ", raw_data_Recovered2.shape)


raw_data_confirmed2.head()

The Shape of Cornfirmed is:  (28143, 6)
The Shape of Cornfirmed is:  (28143, 6)
The Shape of Cornfirmed is:  (28143, 6)


,Province/State,Country/Region,Lat,Long,Date,value
0,NaN,Thailand,15.0000,101.0000,1/22/20,2
1,NaN,Japan,36.0000,138.0000,1/22/20,2
2,NaN,Singapore,1.2833,103.8333,1/22/20,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0


In [62]:
raw_data_confirmed2['Date'] = pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date'] = pd.to_datetime(raw_data_deaths2['Date'])
raw_data_Recovered2['Date'] = pd.to_datetime(raw_data_Recovered2['Date'])

In [63]:
# Renaming the Values
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_Recovered2.columns = raw_data_Recovered2.columns.str.replace('value', 'Recovered')

In [64]:
# Investigating the NULL values
raw_data_confirmed2.isnull().sum()

Province/State    9027
Country/Region       0
Lat                  0
Long                 0
Date                 0
Confirmed            0
dtype: int64

In [65]:
# Dealing with NULL values
raw_data_confirmed2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_deaths2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_Recovered2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)

raw_data_confirmed2.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
Confirmed         0
dtype: int64

In [66]:
# printing shapes before the join
print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered2.shape)

The Shape of Cornirmed is:  (28143, 6)
The Shape of Cornirmed is:  (28143, 6)
The Shape of Cornirmed is:  (28143, 6)


In [67]:
# Full Joins

# Confirmed with Deaths
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'outer', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

# full join with Recovered
full_join = full_join.merge(raw_data_Recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'outer', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.head()

Shape of first join:  (28143, 7)
Shape of second join:  (28261, 8)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2.0,0.0,0.0
1,Japan,Japan,36.0000,138.0000,2020-01-22,2.0,0.0,0.0
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0.0,0.0,0.0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0.0,0.0,0.0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0.0,0.0,0.0


In [68]:
# checking for null values (especially long and lat)
full_join.isnull().sum()

Province/State      0
Country/Region      0
Lat               118
Long              118
Date                0
Confirmed         118
Deaths            118
Recovered         118
dtype: int64

In [69]:
# Adding Month and Year as a new Column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [70]:
# Adding Month and Year as a new Column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [71]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2.0,0.0,0.0,Jan-2020
1,Japan,Japan,36.0000,138.0000,2020-01-22,2.0,0.0,0.0,Jan-2020
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0.0,0.0,0.0,Jan-2020
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0.0,0.0,0.0,Jan-2020
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0.0,0.0,0.0,Jan-2020


In [72]:
#############################################################################################
######################## Braking the numbers by Day #########################################
#############################################################################################

# filtering data to Anhui to give you an example

#creating a new df    
test = full_join[full_join['Province/State'] == 'Anhui']

#creating a new df    
full_join2 = test.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = test.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'outer',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']


test.head()
full_join2.head()
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily
0,Anhui,China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN
1,Anhui,China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020,1.0,0.0,0.0,2020-01-23,2020-01-22,8.0
2,Anhui,China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020,9.0,0.0,0.0,2020-01-24,2020-01-23,6.0
3,Anhui,China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020,15.0,0.0,0.0,2020-01-25,2020-01-24,24.0
4,Anhui,China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020,39.0,0.0,0.0,2020-01-26,2020-01-25,21.0


In [73]:
test.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year
162,Anhui,China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020
639,Anhui,China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020
1116,Anhui,China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020
1593,Anhui,China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020
2070,Anhui,China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020


In [74]:
full_join2.head()

,Province/State,Country/Region,Lat,Long,Date Minus 1,Confirmed - 1,Deaths - 1,Recovered - 1,Month-Year,Date - 1
162,Anhui,China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020,2020-01-23
639,Anhui,China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020,2020-01-24
1116,Anhui,China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020,2020-01-25
1593,Anhui,China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020,2020-01-26
2070,Anhui,China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020,2020-01-27


In [75]:
#############################################################################################
######################## Braking the numbers by Day #########################################
#############################################################################################

## Applying it on all dataset

#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

full_join3.head()

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(28261, 17)


In [76]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Japan,Japan,36.0000,138.0000,2020-01-22,2.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [77]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

In [78]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2.0,0.0,0.0,Jan-2020,2.0,0.0,0.0
1,Japan,Japan,36.0000,138.0000,2020-01-22,2.0,0.0,0.0,Jan-2020,2.0,0.0,0.0
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0


In [79]:
# Exporting the data

# Setting my path
path = "/Users/datascientistenthusiast/Documents/coronavirus/COVID-19/csse_covid_19_data/csse_covid_19_time_series"

# Changing my CWD
os.chdir(path)

#full_join.to_csv('CoronaVirus Tableau Raw', sep='\t')
full_join3.to_csv('CoronaVirus Tableau Raw', sep='\t')

In [80]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Thailand,Thailand,15.0000,101.0000,2020-01-22,2.0,0.0,0.0,Jan-2020,2.0,0.0,0.0
1,Japan,Japan,36.0000,138.0000,2020-01-22,2.0,0.0,0.0,Jan-2020,2.0,0.0,0.0
2,Singapore,Singapore,1.2833,103.8333,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
3,Nepal,Nepal,28.1667,84.2500,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
4,Malaysia,Malaysia,2.5000,112.5000,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
